#### Project Description

Solar energy plays a crucial role in reducing greenhouse gas emissions and meeting rising energy demands. Information on building roof types and geometry is essential for assessing solar potential, improving energy efficiency, and supporting energy transition policies. Roof material and shape data also can inform studies on thermal efficiency, roof durability, disaster risk, heritage conservation, urban heat, and more.

This study aims to develop two ready-to-use, AI-ready open datasets containing both building roof types and geometry, along with a scalable pipeline for roof material classification. The dataset is created using OpenStreetMap (OSM) roof material labels and high-resolution satellite imagery obtained from [OpenAerialMap](https://map.openaerialmap.org/#/7.164459228515626,50.84562199133437,12/square/12020303222/617970f8800b10000509eda7?_k=h73n1n), originally from the Maxar [Open Data Program](https://www.maxar.com/open-data), leveraging OSM building outlines to avoid computationally expensive segmentation processes. This notebook includes the code to obtain and preprocess this data, which can be used to transform any geospatial dataset into the YOLO dataset format, in addition to training a model.

To demonstrate the dataset’s applicability, a convolutional neural network (CNN) model, specifically, Ultralytics' YOLOv11, is used to classify roofs into categories such as roof tiles, tar paper, and metal, followed by an evaluation of the model’s performance.

#### Step 0: Ensure dependencies

In [ ]:
%pip install -r requirements.txt

In [1]:
import os
import yaml
import numpy as np
import geopandas as gpd
import matplotlib as plt
import rasterio
from rasterio.windows import Window
from shapely.geometry import box, Polygon
import cv2
import overpy
from ultralytics import YOLO

#### Step 1: Data Collection

First, we query all the roofs that have a roof:material OpenStreetMap(OSM) tag in the study area. We will be using the Overpass API, which allows us to fetch data from OSM. More information about the Overpass API query language and other examples can be found here: https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_API_by_Example.

We recommend using taginfo's regional databases to get a better understanding of where OpenStreetMap Data you are looking for exists https://taginfo.geofabrik.de/. 

In [5]:
import overpy

def query_osm(bbox: tuple[float, float, float, float]) -> list[dict[str, int|str|list[tuple[float, float]]]]:
    """
    Queries OpenStreetMap (OSM) using Overpass API for buildings within a given bounding box
    that have roof material data.

    Args:
        bbox (tuple[float, float, float, float]): Bounding box coordinates in the format 
            (min_latitude, min_longitude, max_latitude, max_longitude).

    Returns:
        list[dict[str, int | str | list[tuple[float, float]]]]: A list of dictionaries where each dictionary represents a building.
            Each dictionary contains:
            - "id" (int): OSM way ID.
            - "roof_material" (str): Roof material type or "unknown" if not specified.
            - "geometry" (list[tuple[float, float]]): List of (longitude, latitude) tuples 
              representing the building's outline.
    """
    api = overpy.Overpass()
    query = f"""
[out:json][timeout:180];
(
way["building"]["roof:material"]{bbox};
);
out body;
>;
out skel qt;
"""

    response = api.query(query)

    bldgs = []
    for way in response.ways:
        building = {
        "id": way.id,
        "roof_material": way.tags.get("roof:material", "unknown"),
        "geometry": [(float(node.lon), float(node.lat)) for node in way.nodes]
        #shapely expects longitude, latitude, in addition to GeoJSONs, even though WGS894 uses latitude, longitude
    }
        bldgs.append(building)
    return bldgs

# change the bounding box coordinates to get data from your desired study area (min latitude, min longitude, max latitude, max longitude)
bbox = (50.8528358718, 7.09837786378, 50.8568310836, 7.1090734608)
# Cologne: 49.97138, 6.69497, 51.02720390596486, 6.907890818310556
# Bonn: 50.47044, 7.04552, 50.917936, 7.271421
# Meckenheim: 50.13682, 6.90614, 50.5464, 7.0739
# Sample: 50.8528358718, 7.0983778637, 50.8568310836, 7.1090734608
buildings_material = query_osm(bbox)

print(f"total buildings queried: {len(buildings_material)}")
print(f"first 4 buildings: {buildings_material[0:3]}")

total buildings queried: 543
first 4 buildings: [{'id': 77644686, 'roof_material': 'roof_tiles', 'geometry': [(7.1014851, 50.8568545), (7.101623, 50.8568603), (7.1016302, 50.8568606), (7.1016321, 50.8568421), (7.1016359, 50.8568062), (7.1014702, 50.8567992), (7.1014645, 50.8568536), (7.1014851, 50.8568545)]}, {'id': 77644695, 'roof_material': 'roof_tiles', 'geometry': [(7.1019162, 50.8559073), (7.1019143, 50.8559339), (7.1019084, 50.8560152), (7.1020193, 50.8560183), (7.1021328, 50.8560216), (7.1021406, 50.8559137), (7.1019162, 50.8559073)]}, {'id': 77644703, 'roof_material': 'roof_tiles', 'geometry': [(7.1023742, 50.8562211), (7.1023659, 50.8563396), (7.1025967, 50.856346), (7.1026031, 50.8562539), (7.1026049, 50.8562275), (7.1023742, 50.8562211)]}]


Next, we convert the queried OSM data into a usable GeoJSON format

In [6]:
def create_building_material_gdf(buildings):
    """
    Creates a GeoDataFrame from a list of building data, standardizes roof material labels,
    filters out rare categories, and saves the data as GeoJSON files.

    Args:
        buildings (list[dict[str, int | str | list[tuple[float, float]]]]): A list of dictionaries representing buildings,
            each containing:
            - "id" (int): The OSM way ID.
            - "roof_material" (str): The type of roof material.
            - "geometry" (list[tuple[float, float]]): List of (longitude, latitude) tuples.

    Returns:
        gpd.GeoDataFrame: Filtered GeoDataFrame containing buildings with more common roof materials.
    """
    geometries = [Polygon(building["geometry"]) for building in buildings]
    roof_materials = [building["roof_material"] for building in buildings]

    gdf = gpd.GeoDataFrame({"roof_material": roof_materials}, geometry=geometries)
    gdf.set_crs(epsg=4326, inplace=True)

    # filter roof categories to ensure 
    category_counts = gdf["roof_material"].value_counts()
    keep_materials = category_counts[category_counts > 10].index.tolist()
    gdf_filtered = gdf[gdf['roof_material'].isin(keep_materials)]

    output_file = "sample_buildings.geojson"
    gdf.to_file(output_file, driver="GeoJSON")

    output_file_filtered = "sample_buildings_filtered.geojson"
    gdf_filtered.to_file(output_file_filtered, driver="GeoJSON")
    return gdf_filtered

gdf_filtered = create_building_material_gdf(buildings_material)
print(gdf_filtered['roof_material'].value_counts())


roof_material
roof_tiles    435
tar_paper     102
Name: count, dtype: int64


#### Step 2: Data Preprocessing and Formatting
In this step, we use the RoofMaterialDatasetConverter class to prepare our dataset for training a YOLO model. This involves slicing the input raster image into overlapping patches and generating corresponding label files based on the roof material polygons from the GeoJSON. Each patch is filtered to ensure it contains a minimum number of labeled buildings. For valid patches, the building footprints are clipped, normalized, and saved in YOLO segmentation format. The dataset is then automatically split into training, validation, and test sets, and saved in a structured format. A YAML configuration file is also created to define the dataset structure and class labels for use during model training.

Note that the sample dataset is extremely unbalanced and does not include the other 4 notable roof classes. After saving the queried OpenStreetMap data to a GeoJSON, we manually aligned the data and fixed incorrect labels in QGIS. For the purpose of the sample dataset demonstration, we will use the GeoJSON that was just generated.

In [7]:
class RoofMaterialDatasetConverter:
    def __init__(self, 
                 geojson_path: str, 
                 raster_path: str, 
                 output_dir: str, 
                 patch_size: int = 640, 
                 overlap: int = 128,
                 patch_min_buildings: int = 2) -> None:

        self.gdf = gpd.read_file(geojson_path)
        self.raster = rasterio.open(raster_path)
        
        self.output_dir = output_dir
        
        for split in ['train', 'val', 'test']:
            os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
            os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)
        
        self.patch_size = patch_size
        self.overlap = overlap
        self.patch_min_buildings = patch_min_buildings

        self.class_mapping = {
            'roof_tiles': 0,
            'tar_paper': 1,
            'metal': 2,
            'concrete': 3,
            'tile': 4,
            'gravel': 5,
            'glass': 6
        }

    def generate_patches(self, 
                     train_ratio: float = 0.7, 
                     val_ratio: float = 0.15, 
                     random_state: int = 42) -> dict[str, list[tuple[str, str]]]:
        """
        Generate image patches with corresponding roof material labels.
        
        Returns:
            list of tuples: [(image_path, label_path), ...]
        """
        rng = np.random.default_rng(random_state)

        # collect all potential patches
        all_patches: list[tuple[str, str]] = []
        
        height, width = self.raster.height, self.raster.width
        for y in range(0, height, self.patch_size - self.overlap):
            for x in range(0, width, self.patch_size - self.overlap):
                window = Window(x, y, self.patch_size, self.patch_size)
                
                # read patch from raster
                try:
                    patch = self.raster.read(window=window)
                    patch = np.transpose(patch[:3], (1, 2, 0))  # transpose to HWC
                except Exception as e:
                    print(f"Skipping patch at ({x},{y}) due to error: {e}")
                    continue
                
                # check if patch contains any roof material labels
                patch_bounds = box(
                    self.raster.xy(y, x)[0], 
                    self.raster.xy(y, x)[1],
                    self.raster.xy(y + self.patch_size, x + self.patch_size)[0],
                    self.raster.xy(y + self.patch_size, x + self.patch_size)[1]
                )
                
                # filter roof polygons within this patch
                patch_roofs = self.gdf[self.gdf.intersects(patch_bounds)]
                
                if len(patch_roofs) >= self.patch_min_buildings:
                    label_content = self._generate_labels(patch_roofs, patch_bounds)
                    if label_content:
                        all_patches.append((patch, (y, x), label_content))
        
        # randomly split patches
        n_patches = len(all_patches)
        train_size = int(n_patches * train_ratio)
        val_size = int(n_patches * val_ratio)
        
        rng.shuffle(all_patches)
        
        train_patches = all_patches[:train_size]
        val_patches = all_patches[train_size:train_size+val_size]
        test_patches = all_patches[train_size+val_size:]
        
        split_paths = {
            'train': [],
            'val': [],
            'test': []
        }
        
        for split, patches in [('train', train_patches), ('val', val_patches), ('test', test_patches)]:
            for patch, (y, x), label_content in patches:
                patch_filename = f'patch_{y}_{x}.png'
                
                img_path = os.path.join(self.output_dir, 'images', split, patch_filename)
                label_path = os.path.join(self.output_dir, 'labels', split, patch_filename.replace('.png', '.txt'))
                
                os.makedirs(os.path.dirname(img_path), exist_ok=True)
                os.makedirs(os.path.dirname(label_path), exist_ok=True)
                
                cv2.imwrite(img_path, cv2.cvtColor(patch, cv2.COLOR_RGB2BGR))
                
                with open(label_path, 'w') as f:
                    f.write('\n'.join(label_content))
                split_paths[split].append((img_path, label_path))
        
        return split_paths

    def _generate_labels(self, 
                         roof_polygons: gpd.GeoDataFrame,
                         patch_bounds: box) -> list[str]:
        """
        Generate YOLO segmentation labels for roof polygons in a patch.
        
        Parameters:
            roof_polygons : GeoDataFrame
                GeoDataFrame of roof polygons in the current patch
            patch_bounds : shapely.geometry.box
                Bounding box of the current patch
        
        Returns:
            list of label strings
        """
        labels: list[str] = []
        
        for _, roof in roof_polygons.iterrows():
            # only include specified roof material classes
            class_name = roof['roof_material']
            if class_name not in self.class_mapping:
                continue
            class_index = self.class_mapping[class_name]
            
            # clip polygon to patch bounds
            clipped_roof = roof.geometry.intersection(patch_bounds)
            
            if not clipped_roof.is_empty:
                try:
                    coords = np.array(clipped_roof.exterior.coords)
                    normalized_coords = self._normalize_coordinates(coords, patch_bounds)
                    
                    if len(normalized_coords) >= 3:
                        # create label string: class_index x1 y1 x2 y2 ...
                        label = f"{class_index} " + " ".join(map(str, normalized_coords.flatten()))
                        labels.append(label)
                except Exception as e:
                    print(f"Error processing roof polygon: {e}")
        
        return labels
    
    def _normalize_coordinates(self, 
                               coords: np.ndarray, 
                               patch_bounds: box) -> np.ndarray:
        """
        Normalize polygon coordinates to 0-1 range within patch.
        
        Parameters:
            coords: numpy.ndarray
                Original polygon coordinates
            patch_bounds : shapely.geometry.box
                Bounding box of the current patch
        
        Returns:
            numpy.ndarray of normalized coordinates
        """
        x_min, y_min = patch_bounds.bounds[0], patch_bounds.bounds[1]
        x_max, y_max = patch_bounds.bounds[2], patch_bounds.bounds[3]
        normalized_coords = coords.copy()
        normalized_coords[:, 0] = (coords[:, 0] - x_min) / (x_max - x_min)
        #normalized_coords[:, 1] = (coords[:, 1] - y_min) / (y_max - y_min)
        normalized_coords[:, 1] = (y_max - coords[:, 1]) / (y_max - y_min)

        return normalized_coords
    
    def create_dataset_yaml(self) -> str:
        """
        Create YOLO dataset configuration YAML file.
        
        Returns:
            str: Path to the created YAML file
        """
        # reverse order for right YAML formatting
        class_names = {v: k for k, v in self.class_mapping.items()}
        
        yaml_content = {
            'path': self.output_dir,
            'train': 'images/train',
            'val': 'images/val',
            'test': 'images/test',
            'names': class_names
        }
        
        yaml_path = os.path.join(self.output_dir, 'roof_materials.yaml')
        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_content, f, default_flow_style=False)
        
        return yaml_path


In [25]:
geojson_path = 'sample_buildings.geojson'
raster_path = '61796764800b10000509eda2.tif'
output_dir = 'datasets/sample_dataset'

converter = RoofMaterialDatasetConverter(
    geojson_path, 
    raster_path, 
    output_dir,
    patch_min_buildings=3
)

dataset_splits = converter.generate_patches(
    train_ratio=0.7,
    val_ratio=0.15,
    random_state=42
)

yaml_path = converter.create_dataset_yaml()

print(f"YOLO dataset created at {output_dir}")
print(f"Train patches: {len(dataset_splits['train'])}")
print(f"Validation patches: {len(dataset_splits['val'])}")
print(f"Test patches: {len(dataset_splits['test'])}")
print(f"Dataset YAML created at {yaml_path}")


YOLO dataset created at datasets/sample_dataset
Train patches: 11
Validation patches: 2
Test patches: 3
Dataset YAML created at datasets/sample_dataset/roof_materials.yaml


#### Step 3: Model Training
We train a YOLOv8 segmentation model (yolo11n-seg.pt) on a custom dataset of roof materials. Training runs for up to 100 epochs. The model uses 640x640 images (or any multiple of 16 for the patch size), a batch size of 16, and leverages a GPU (device='0'). Training plots are enabled for visualizing performance metrics. Actual model results will be attached.

In [ ]:
model = YOLO('yolo11n-seg.pt')


results = model.train(
    data="datasets/sample_dataset/roof_materials.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    plots=True,
    device='0',
    pretrained=True,
    patience=20,
    save_period=10
)

#### Step 4: Model Inference
We will apply the model we just trained to label other images (to be completed).

In [ ]:
inference_results = model.predict('', save=True, imgsz=640)

#### Next Steps + Potential Improvements

- Experimenting with smaller patch sizes to decrease the amount of patches with unlabelled houses
- Applying data augmentation to balance out the class imbalance
- Creating a pipeline that works for areas with sparse/non-grouped labels such as Nepal, Mozambique, Sri Lanka, and India. This was the initial goal but we realized data quality was subpar, so we pivoted to creating a reproducible pipeline in a data-rich area that could later be applied to other areas.
- Elaborate more in the documentation and provide clearer step descriptions
- Creating a pipeline that works for medium-high resolution satellite imagery
- Evaluation and inference on nearby areas in Germany to expand available data
- Manually validating test set data
